<a href="https://colab.research.google.com/github/napoles-uach/ML-FCQ/blob/main/keras_numeros_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q

In [5]:
%%writefile app.py
import streamlit as st
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd


st.title('Keras example')
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

images_test = test_images

model = keras.Sequential([
 layers.Dense(512, activation="relu"),
 layers.Dense(10, activation="softmax")
])
code = '''model = keras.Sequential([
 layers.Dense(512, activation="relu"),
 layers.Dense(10, activation="softmax")
])'''
st.code(code, language='python')

model.compile(optimizer="rmsprop",
 loss="sparse_categorical_crossentropy",
 metrics=["accuracy"])
code_comp = '''model.compile(optimizer="rmsprop",
 loss="sparse_categorical_crossentropy",
 metrics=["accuracy"])
'''
st.code(code_comp,language='python')

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

#model.fit(train_images, train_labels, epochs=5, batch_size=128)
#@st.experimental_singleton
def model_fit(model):
  history = model.fit(train_images, train_labels, epochs=5, batch_size=128)
  return history
code_fit = 'model.fit(train_images, train_labels, epochs=5, batch_size=128)'
st.code(code_fit,language='python')

train = st.checkbox('train')
if train:
  history = model_fit(model)
  df = pd.DataFrame(history.history)
  st.line_chart(df)




test_digits = test_images[0:10]
predictions = model.predict(test_digits)
n=st.sidebar.selectbox("Indice",[0,1,2,3,4,5,6,7,8,9])
fig, ax = plt.subplots()
im = ax.imshow(images_test[n], cmap="binary")
st.sidebar.pyplot(fig)

st.header("Prediccion: "+str(predictions[n].argmax()))
st.sidebar.write("Valor verdadero "+str(test_labels[n]))

test_loss, test_acc = model.evaluate(test_images, test_labels)
st.write(f"test_acc: {test_acc}")
st.write(model.summary())

Overwriting app.py


In [3]:
!streamlit run app.py & npx localtunnel --port 8501

2022-10-09 22:21:56.670 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
[###########.......] / extract:yargs-parser: sill extract yargs-parser@^20.2.2

npx: installed 22 in 4.289s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.15.123:8501

your url is: https://open-onions-appear-34-125-15-123.loca.lt
2022-10-09 22:36:18.567067: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 0.2538 - accuracy: 0.9267
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1023 - accuracy: 0.9698
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0681 - accuracy: 0.9792
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0490 - accuracy: 0.9855
Epoch 5/5
469/469 [==============================] - 4